## Store/retrieve rows from a Dataframe from disk

The goal of a `TableIndexer` is two-fold:
- Use `.index(df, cols)` to index some user defined columns of a table (pd.DataFrame) to disk.
- Use `.search(query)` to retrieve the rows of the dataframe that match the `query`

In [21]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from lsearch import TableIndexer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df_ = pd.read_parquet('dataset.parquet')

In [32]:
df_[0:2]

,parent_asin,main_category,title,description,features,embellished_description,brand,images,short_query,long_query
0,B016UIUCRG,Sports & Outdoors,NHL Molded Auto Emblem,Showcase your team spirit with this eye-catchi...,Made in US\nEye-catching auto emblem measures ...,###Description###\nShowcase your team spirit w...,Rico Industries,[{'hi_res': 'https://m.media-amazon.com/images...,NHL auto emblem sticker,3D molded NHL auto emblem sticker for Washingt...
1,B01ARINE7U,AMAZON FASHION,Dickies Men's Short Sleeve Western Shirt,Dickies is the leading Global brand of innovat...,"55% Cotton, 45% Polyester\nImported\nMachine W...",###Description###\nThis Dickies Men's Short Sl...,Dickie's Men's Tops,[{'hi_res': 'https://m.media-amazon.com/images...,Dickies Men's Short Sleeve Western Shirt,Dickies men's plaid short sleeve western shirt...


In [47]:
cols = ["parent_asin", 
        "main_category",
        "title",
        "description",
        "features", 
        "brand",
        "short_query"]

schema = {col:'str' for col in cols}
schema

{'parent_asin': 'str',
 'main_category': 'str',
 'title': 'str',
 'description': 'str',
 'features': 'str',
 'brand': 'str',
 'short_query': 'str'}

In [64]:
df_indexer = TableIndexer(bin_path = "./table_indexer", 
                          schema=schema,
                          variable_length_columns = cols)

In [65]:
index_cols = ['title', 'description']
df_indexer.index(df=df_, index_cols=index_cols, metadata_cols=cols, path='./table_indexer_')

InvertedIndex stored in ./table_indexer_/inv_index_title
InvertedIndex stored in ./table_indexer_/inv_index_description


In [67]:
%%time
df_indexer.search('nintendo switch')

CPU times: user 12.6 ms, sys: 1.39 ms, total: 14 ms
Wall time: 13.7 ms


,parent_asin,main_category,title,description,features,brand,short_query
0,B0BMQ8FPCQ,AMAZON FASHION,"4pcs Back Stand for Nintendo Switch, Back Stan...",Package Includes: 4 x Kickstand Replacement(Bl...,Premium Material: The kickstand replacements a...,Flohayo,kickstand replacement for Nintendo Switch
1,B0BQJ3R35N,Sports & Outdoors,SOAR NFL LED Gaming Headset and Stand,The SOAR RGX LED gaming headphones and stand p...,Imported\nRGB Color Changing LED Lights on Hea...,SOAR,SOAR NFL LED Gaming Headset
2,B07ZBPMYBR,AMAZON FASHION,Plus Size Adult Deluxe Donkey Kong Costume,"Donkey Kong really has the life, doesn't he? O...",100% Polyester\nImported\nNo Closure closure\n...,Disguise,Donkey Kong costume
3,B07FTG3D2S,AMAZON FASHION,Tennis Racket for Nintendo Switch Joy-Con Cont...,Product description\nRuiao Durable & High Qual...,<p>Compatible with Nintendo Switch Game- Mario...,Ruiao,Tennis Racket for Nintendo Switch


## Load inverted index from folder

In [11]:
inv_index_from_disk = InvertedIndex.read_inv_index('./inv_index_store')

In [12]:
inv_index_from_disk.__dict__.keys()

dict_keys(['word2pos', 'n_docs_seen', 'folder_store', 'word_freq', 'doc_freq', 'postings_term_pointers'])

In [13]:
word_pos_tuples = inv_ind.word2pos.items()
words = [x[0] for x in word_pos_tuples]
pos = [x[1] for x in word_pos_tuples]

inv_ind.write_strings_to_file("word2pos_test.bin", inv_ind.word2pos)

In [14]:
vocab_from_file = inv_ind.read_strings_from_file("word2pos_test.bin")

In [15]:
vocab_from_file[0:10]

['from', 'lerxst', 'wam', 'umd', 'edu', 'where', 's', 'my', 'thing', 'subject']

## Make queries (single term) from the loaded indexer

In [16]:
%%timeit
inv_index_from_disk.get_tuples_for_term_id(12312)

90.5 µs ± 771 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%timeit
inv_index_from_disk.get_tuples_for_term_id_slow(12312)

1.55 ms ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
inv_index_from_disk.get_tuples_for_term_id(12312)

[(12312, 227, 1),
 (12312, 954, 1),
 (12312, 1672, 1),
 (12312, 2721, 1),
 (12312, 3568, 1),
 (12312, 7209, 1)]

In [19]:
inv_index_from_disk.get_tuples_for_term_id_slow(12312)

[(12312, 227, 1),
 (12312, 954, 1),
 (12312, 1672, 1),
 (12312, 2721, 1),
 (12312, 3568, 1),
 (12312, 7209, 1)]

We can use a `term` (string) instead of the `term_id` (integer).

In [20]:
inv_index_from_disk.get_tuples_for_term('nintendo')

[(23577, 525, 3),
 (23577, 830, 2),
 (23577, 1125, 2),
 (23577, 2138, 1),
 (23577, 2839, 1),
 (23577, 3867, 1),
 (23577, 4922, 1),
 (23577, 5855, 1),
 (23577, 6016, 2),
 (23577, 6470, 1),
 (23577, 7890, 1),
 (23577, 8815, 1),
 (23577, 9023, 2),
 (23577, 9382, 2),
 (23577, 9941, 3)]

## Storing list of strings to disk

In [21]:
vocab = list(inv_index_from_disk.word2pos.keys())

In [22]:
%%timeit
inv_index_from_disk.write_strings_to_file('test.bin', vocab)

35.8 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
v = inv_index_from_disk.read_strings_from_file('test.bin')

7.55 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Make queries (multiple terms) from the loaded indexer


The notebook 

```
chap_6_processing_boolean_queries.ipynb
```

has a detailed method to compute intersections between postings that are sorted by `doc_id`.

In [24]:
query = 'nintendo super mario'
inv_index_from_disk.search_postings_for_terms(query)

[[(23577, 525, 3),
  (23577, 830, 2),
  (23577, 1125, 2),
  (23577, 2138, 1),
  (23577, 2839, 1),
  (23577, 3867, 1),
  (23577, 4922, 1),
  (23577, 5855, 1),
  (23577, 6016, 2),
  (23577, 6470, 1),
  (23577, 7890, 1),
  (23577, 8815, 1),
  (23577, 9023, 2),
  (23577, 9382, 2),
  (23577, 9941, 3)],
 [(5259, 70, 1),
  (5259, 173, 2),
  (5259, 525, 2),
  (5259, 830, 1),
  (5259, 1125, 2),
  (5259, 1331, 1),
  (5259, 1395, 1),
  (5259, 1401, 1),
  (5259, 1461, 1),
  (5259, 1624, 3),
  (5259, 1822, 1),
  (5259, 2179, 1),
  (5259, 2194, 1),
  (5259, 2269, 1),
  (5259, 2303, 1),
  (5259, 2316, 2),
  (5259, 2379, 1),
  (5259, 2526, 1),
  (5259, 2665, 1),
  (5259, 2676, 1),
  (5259, 2705, 1),
  (5259, 2741, 1),
  (5259, 2749, 1),
  (5259, 3024, 1),
  (5259, 3169, 2),
  (5259, 3179, 1),
  (5259, 3209, 1),
  (5259, 3226, 1),
  (5259, 3243, 1),
  (5259, 3480, 4),
  (5259, 3574, 1),
  (5259, 3696, 1),
  (5259, 3777, 1),
  (5259, 3822, 3),
  (5259, 3867, 1),
  (5259, 3869, 1),
  (5259, 3966, 1),
  (

In [25]:
%%timeit
inv_index_from_disk.search(query)

268 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
search_results = inv_index_from_disk.search(query)
search_results

[830, 9023]

In [27]:
print(docs[search_results[1]])

From: fields@cis.ohio-state.edu (jonathan david fields)
Subject: Misc. Stuff for Sale
Article-I.D.: penguin.1po5lqINN749
Distribution: usa
Organization: The Ohio State University Dept. of Computer and Info. Science
Lines: 46
NNTP-Posting-Host: penguin.cis.ohio-state.edu


Misc. Items for sale:


Walkman:  Aiwa Model HS-T30, 1 year old, mint condition, hardly used, 
          autoreverse, 3 band Equalizer, Super Bass, Dolby Noise Reduction,
          AM FM tuner..........Paid $70.......Asking $40+shipping.

Mount Plate:  Sony Model CPM-203P, mounting plate for Sony portable CD players
for Portable: plugs into car lighter, snaps onto the bottom of any Sony
CD Player:    Portable CD player, perfect condition. Will also throw in a 
	      cassette adapter in SO SO condition.
	      Paid $45...............Asking $30+shipping.

AM FM:	    Factory Stereo from Toyota with AM FM radio, autoreverse cassette
Cassette:   deck, digital tuning, like new condition only in car 6 months,
Car Stereo: As

### How can we store metadata in the inverted index and return it when the user calls .search?

In [28]:
from datasets import load_from_disk

train_dataset = load_from_disk("/Users/dbuchaca/Desktop/text_retrieval_and_search_engines/datasets/amazon_2023_genq/train_dataset")
# If running from QNAP 
#train_dataset = load_from_disk("/Users/davidbuchaca1/Datasets/amazon_2023_genq/train_dataset")
train_dataset

Dataset({
    features: ['parent_asin', 'main_category', 'title', 'description', 'features', 'embellished_description', 'brand', 'images', 'short_query', 'long_query'],
    num_rows: 205637
})

In [29]:
df = train_dataset.to_pandas()
df.head()
df.to_parquet('./dataset.parquet')

Read as parquet

In [30]:
df_ = pd.read_parquet('dataset.parquet')

Storing metadata for documents as an external parquet file such as https://chatgpt.com/c/68080f54-97c0-8008-8c4f-7de93aa43048

## Row serialization process for dataframe

In [32]:
from lsearch import TableSerializer

In [33]:
cols = ["parent_asin", 
        "main_category",
        "title",
        "description",
        "features", 
        "brand",
        "short_query"]

schema = {col:'str' for col in cols}
schema

{'parent_asin': 'str',
 'main_category': 'str',
 'title': 'str',
 'description': 'str',
 'features': 'str',
 'brand': 'str',
 'short_query': 'str'}

In [34]:
table_ser = TableSerializer(bin_path="variable_bin_storage.bin", 
                            schema=schema,
                            variable_length_columns = cols)

In [35]:
%%time
table_ser.serialize(df_)

CPU times: user 23.5 s, sys: 1.83 s, total: 25.3 s
Wall time: 25.9 s


In [36]:
%%timeit
table_ser.read_rows_parallel([0, 520, 40])

1.05 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Retrieve serialized rows for a query search

In [37]:
(df_['title'] + ' ' + df_['description']).iloc[0]

'NHL Molded Auto Emblem Showcase your team spirit with this eye-catching Chrome Finished Auto Emblem 3D Sticker by Rico Industries. This auto emblem 3D sticker measures 5-inches by 5-inches and is decorated with a dynamic and bold team logo. It easily adheres to any vehicle or other hard surface and is made of weather resistant materials. Made in the USA.'

In [39]:
%%time
inv_ind = InvertedIndex()
inv_ind.index(df_['title'] + ' ' + df_['description'], folder_store="inv_index_store_amazon_2023_genq")

InvertedIndex stored in inv_index_store_amazon_2023_genq
CPU times: user 49.3 s, sys: 4.15 s, total: 53.4 s
Wall time: 55.5 s


In [40]:
query = 'nintendo super mario'

In [41]:
%%time
search_results = inv_ind.search(query)
table_ser.read_rows_parallel(search_results)

CPU times: user 11.1 ms, sys: 8.55 ms, total: 19.6 ms
Wall time: 27.8 ms


,parent_asin,main_category,title,description,features,brand,short_query
0,B08JZMQMNF,AMAZON FASHION,"Super Mario Nintendo Rain Boots,Mid Height Sli...","Stomping, hopping, and walking in the rain has...",Rubber sole\nSUPER MARIO FUN: Super Mario Bros...,SUPER MARIO,Super Mario Rain Boots for Kids
1,B09TRWSQYG,AMAZON FASHION,Nintendo Super Mario Little & Big Boys Swim Tr...,Little and big kids will love these Super Mari...,100% Polyester\nImported\nElastic closure\nSwi...,Nintendo,Super Mario boys swim trunks
2,B07ZZLH18W,AMAZON FASHION,Mario Kart Nintendo Boys' Super Mario Drifting...,This tee is a special edition not available an...,100% Cotton\nPull On closure\nMachine Wash\nLO...,Mario Kart,Mario Kart t-shirt for boys
3,B0BXVXKFLQ,AMAZON FASHION,Nintendo Boys' Super Mario Boxer Briefs Availa...,All packs available are fun and adds a unique ...,"92% Polyester, 8% Spandex\nImported\nPull On c...",Handcraft Children's Apparel,Nintendo Boxer Briefs
4,B07HYBLKRD,AMAZON FASHION,Jumping Beans Boys 4-10 Nintendo Super Mario B...,Boys 4-10 Jumping Beans Nintendo Super Mario B...,100% Cotton\nPull On closure\nMachine Wash\nCr...,Jumping Beans,Boys Yoshi Graphic Tee
5,B06X97PPY7,AMAZON FASHION,Nintendo Boy's Super Mario Groupage T-Shirt,Bring your favorite video game friends everywh...,100% Cotton\nMade in the USA and Imported\nNo ...,Fifth Sun,Nintendo Mario T-Shirt
6,B06X6NLS7J,AMAZON FASHION,Nintendo Girl's More Fun T-Shirt,Join all your favorite friends from the super ...,"60% Cotton, 40% Polyester\nMade in the USA and...",Fifth Sun,Nintendo Girl's T-Shirt
7,B089B6QFQM,AMAZON FASHION,"Nintendo Baseball Cap, Super Mario Adjustable ...",He'll get his jumping powers started with this...,100% Cotton\nImported\nAdjustable closure\nHan...,Nintendo,Nintendo Baseball Cap
8,B08H9HBT72,AMAZON FASHION,SUPER MARIO Boys' Nintendo 2-Piece Fleece Paja...,To follow safety regulations this item is flam...,100% Polyester\nImported\nPull On closure\nMac...,AME Sleepwear Children's Apparel,Super Mario Boys Pajama Set
9,B08WPJ6GCH,AMAZON FASHION,Super Mario Brothers Mario & Luigi Nintendo Sa...,Officially Super Mario Brothers Sandals – Boys...,These officially licensed Super Mario Brothers...,Super Mario Bros.,Super Mario Sandals


In [42]:
len(search_results)

47